# Notebook 10 - Quantum QHBM / VQT Tutorial

**Level:** Advanced  
**Topics:** Quantum Hamiltonian-based models, variational quantum thermalization, free energy

> **Provenance:** This notebook is based on:
> - `light_theory_realm/models/quantum_qhbm.py`
> - `light_theory_realm/models/README.md` quantum examples


In [ ]:

import math
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

try:
    import pennylane as qml
    from light_theory_realm.models.quantum_qhbm import (
        LatentDistribution,
        QuantumLayer,
        vqt_loss,
    )
    HAS_QHBM = True
except Exception as exc:
    HAS_QHBM = False
    print(f"PennyLane/QHBM backend not available, using fallback demo: {exc}")
    qml = None

plt.rcParams["figure.figsize"] = (7, 5)


## 1. A small 2-qubit Ising Hamiltonian

We use a simple Hamiltonian: H = - Z0 Z1 - h (Z0 + Z1) with a small transverse field h.


In [ ]:

if HAS_QHBM:
    def pauli_z():
        return jnp.array([[1.0, 0.0], [0.0, -1.0]])

    def build_ising_hamiltonian(h=0.3):
        coeffs = [-1.0, -h, -h]
        ops = [qml.PauliZ(0) @ qml.PauliZ(1), qml.PauliZ(0), qml.PauliZ(1)]
        return qml.Hamiltonian(coeffs, ops)

    H = build_ising_hamiltonian()
    H
else:
    H = None
    print("Using toy Hamiltonian surrogate (no PennyLane backend).")


## 2. Latent model and quantum layer

We instantiate a tiny latent model and a quantum layer, then define the VQT free-energy objective.


In [ ]:

if HAS_QHBM:
    key = jax.random.PRNGKey(0)
    latent = LatentDistribution(n_qubits=2)
    quantum_layer = QuantumLayer(n_qubits=2, n_layers=2)
    beta = 1.0
    params = {
        "latent": latent.init_params(key),
        "quantum": quantum_layer.init_params(key),
    }
else:
    key = jax.random.PRNGKey(0)
    beta = 1.0
    params = {
        "latent": jnp.array([0.1, -0.1]),
        "quantum": jnp.array([0.2, -0.2]),
    }


## 3. VQT loss and gradient step

We minimize a VQT-style loss: F = beta * E - S, where E is energy and S is an entropy-like term.


In [ ]:

if HAS_QHBM:
    def loss_fn(params, key):
        return vqt_loss(
            theta=params["latent"],
            phi=params["quantum"],
            latent_model=latent,
            quantum_layer=quantum_layer,
            hamiltonian=H,
            beta=beta,
            key=key,
        )
else:
    def loss_fn(params, key):
        # Simple quadratic surrogate
        return jnp.sum(params["latent"] ** 2) + jnp.sum(params["quantum"] ** 2)

grad_loss = jax.jit(jax.grad(loss_fn))

def step(params, key, lr=0.05):
    g = grad_loss(params, key)
    new_params = jax.tree_util.tree_map(
        lambda p, gp: p - lr * gp, params, g
    )
    loss_val = float(loss_fn(new_params, key))
    return new_params, loss_val


> Note: If you have PennyLane + the QHBM backend available, keep `HAS_QHBM=True` and run the full VQT loss. Otherwise the surrogate loss keeps the notebook runnable.


## 4. Run optimization and track free energy


In [ ]:

num_steps = 20
loss_history = []

params_opt = params
key = jax.random.PRNGKey(1)

for t in range(num_steps):
    key, subkey = jax.random.split(key)
    params_opt, loss_val = step(params_opt, subkey, lr=0.05)
    loss_history.append(loss_val)

plt.figure()
plt.plot(loss_history)
plt.xlabel("Step")
plt.ylabel("VQT free energy F" if HAS_QHBM else "Surrogate loss")
plt.title("VQT optimization" if HAS_QHBM else "Surrogate optimization")
plt.grid(True, linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()


## 5. Interpreting the free-energy curve

As training progresses, the VQT objective typically decreases, indicating a better approximation to the target thermal state.

Questions to explore:

- How does changing beta affect convergence?
- What happens if you increase the circuit depth or number of qubits?
- How does the learned state compare to the exact thermal state of the Ising Hamiltonian?

Next steps:

- Export this setup into a script for batch experiments.
- Connect geometric diagnostics (QGT, Berry, xi) to QHBM parameter space trajectories.
